In [25]:
pip install asyncio


     -------------------------------------- 101.8/101.8 kB 6.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
rows = cw.query_produrements()

urls = []
for row in rows:
    url = 'https://{base}/portaltransparencia-api/api/licitacoes/arquivos?entidade=1&exercicio={year}&tipoLicitacao={type}&licitacao={num}'.format(
        base=cw._url,
        year=row[2],
        type=row[3],
        num=row[4]
    )
    urls.append(url)

In [5]:
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

responses = []

async def fetch(client, url):
    async with client.get(url) as resp:
        assert resp.status == 200
        html = await resp.json()
        responses.append(html)


async def main():
    async with aiohttp.ClientSession() as client:
        await asyncio.gather(*[
            asyncio.ensure_future(fetch(client, url))
            for url in urls
        ])


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [6]:
pip install nest-asyncio


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
i=0
to_insert = []
for response in responses:
    i += 1
    for item in response:
        id = str(item['licitacao']) + '/' + str(item['exercicio']) + '/' + str(item['tipoLicitacao'])
        to_insert.append((cw._id_ibge, id, item['arquivoCompras']['idArquivo'], os.path.join("/" + str(cw._id_ibge), id, str(item['arquivoCompras']['idArquivo']) + ".pdf"), item['arquivoCompras']['nomeArquivo'], item['arquivoCompras']['data']))

In [8]:
import sqlite3

In [9]:
insert = "INSERT OR IGNORE INTO FILES (ibge, procurement_id, file_id, path, OCR_RAW, name, date) VALUES (?, ?, ?, ?, '', ?, ?)"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_insert)
con.commit()
con.close()

In [4]:
pip install aiofile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')

Scraping.crawlers.crawler_oxy


In [3]:
rows = cw.query_files(5)
rows

[(4119905, '1/2015/6', 6429),
 (4119905, '1/2015/7', 6715),
 (4119905, '1/2015/9', 6729),
 (4119905, '1/2016/6', 6099),
 (4119905, '1/2016/7', 6139)]

In [9]:
import os
rows = cw.query_files()

i = 1
to_update = []
for row in rows:   
    file = os.path.join("", "Data", row[0].__str__(), row[1].replace("/", "-"))
    to_update.append((path, row[0], row[1], row[2]))

In [10]:
insert = "UPDATE FILES set path = ? where ibge = ? and procurement_id = ? and file_id = ?"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_update)
con.commit()
con.close()

In [18]:
#Validar pastas
pastas = cw.query_folders()
for pasta in pastas:
    if not os.path.exists(pasta[0]):
        os.makedirs(pasta[0]) 

In [18]:
files = cw.query_files(3)
files2 = [('https://{base}/portaltransparencia-api/api/files/arquivo/{file_id}'.format(
                base=cw._url,
                file_id=x[2]
            ), ) + x for x in files]
files2

[('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6429',
  4119905,
  '1/2015/6',
  6429,
  'Data\\4119905\\1-2015-6',
  '6429.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6715',
  4119905,
  '1/2015/7',
  6715,
  'Data\\4119905\\1-2015-7',
  '6715.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729',
  4119905,
  '1/2015/9',
  6729,
  'Data\\4119905\\1-2015-9',
  '6729.pdf')]

In [24]:
for file in files2:
    print(file)

('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729', 4119905, '1/2015/9', 6729, 'Data\\4119905\\1-2015-9', '6729.pdf')


In [16]:
pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
#Check if file exists
files3 = [x for x in files2 if not os.path.isfile(os.path.join(x[4], x[5]))]

In [26]:
#baixar e salvar cada arquivo

import os
import asyncio
import aiohttp  # pip install aiohttp
import aiofile  # pip install aiofile
import nest_asyncio

nest_asyncio.apply()

def download_files_from_report(files):

    #sema = asyncio.BoundedSemaphore(5)

    async def fetch_file(session, file):
        
        url = file[0]
        folder = file[4]
        fname = file[5]

        #async with sema:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

        async with aiofile.async_open(
            os.path.join(folder, fname), "wb"
        ) as outfile:
            await outfile.write(data)

    async def main():
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_file(session, file) for file in files]
            await asyncio.gather(*tasks)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    #loop.close()

download_files_from_report(files3)

In [1]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
try:
    cw.get_all_files()
except:
    pass

Scraping.crawlers.crawler_oxy
there is 2483 files in the database
there is 1827 files to be downloaded
the first file id is: 6139.
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded


IndexError: list index out of range

In [1]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')

Scraping.crawlers.crawler_oxy


In [2]:
file = r'Data\4119905\6429.pdf'
file

'Data\\4119905\\6429.pdf'

In [4]:
pip install pytesseract


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pytesseract
from pytesseract import Output
from PIL import Image
import cv2

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
img_path1 = 'test.png'
text = pytesseract.image_to_string(img_path1,lang='por')
print(text)

Câmara Municipal de Ponta Grossa

Ponta Grossa, 25 de fevereiro de 2015

Senhor Presidente:

Venho pelo presente, respeitosamente, solicitar autorização para
aquisição de MATERIAL DE CONSUMO, em relação anexa, para atender a demanda desta
Câmara Municipal pelo período de 01 (um) ano

Certo de sua atenção, subscrevo-me, reiterando protestos de
consideração e estima

Atenciosamente

/

e moeb O.

RENATO WEBBER DE OLIVEIRA
Diretor Geral de Serviços Administrativos

om

AO EXMO. SR.
SEBASTIÃO MAINARDES JR. —
PRESIDENTE DA CÂMARA MUNICIPAL DE PONTA GROSSA

880 - Ponta Grossa - Pr - CEP 84051-000 - Fon




In [3]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'